In [3]:
#1. LOAD DATASET
import pandas as pd
from ast import literal_eval

# read dataframe from csv on gitlab
url = 'https://gitlab.com/rahasak-labs/dot/-/raw/master/src/main/resources/movie.csv'
df = pd.read_csv(url)

# pandas read list type colomns(genres, cast, keywords) as string, so convert them back to list type
df.genres = df.genres.apply(literal_eval)
df.cast = df.cast.apply(literal_eval)
df.keywords = df.keywords.apply(literal_eval)

# pretty print features dataframe
from tabulate import tabulate
print(tabulate(df[['title', 'cast', 'director', 'keywords', 'genres']].head(5), headers='keys', tablefmt='psql'))

+----+------------------------------------------+--------------------------------------------------------+-------------------+---------------------------------------------+--------------------------------------------+
|    | title                                    | cast                                                   | director          | keywords                                    | genres                                     |
|----+------------------------------------------+--------------------------------------------------------+-------------------+---------------------------------------------+--------------------------------------------|
|  0 | Avatar                                   | ['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver'] | James Cameron     | ['culture clash', 'future', 'space war']    | ['Action', 'Adventure', 'Fantasy']         |
|  1 | Pirates of the Caribbean: At World's End | ['Johnny Depp', 'Orlando Bloom', 'Keira Knightley']    | Gore Verbinski    | [

**Penjelasan kode program 1**
Kode di bagian ini digunakan untuk:
1. Mengambil dataset film dari internet (GitLab) dalam format CSV.
2. Mempersiapkan data agar dapat diproses lebih lanjut, dengan mengubah data teks (string) dalam kolom tertentu menjadi list Python yang bisa dimanipulasi
3. Menampilkan cuplikan data awal (5 baris pertama) dalam format tabel rapi untuk memastikan bahwa data sudah berhasil dimuat dan dikonversi dengan benar.

In [4]:
#2. CLEAN DATA
# convert all strings to lower case and remove the spaces
def clean_feature(x):
    if isinstance(x, list):
        # apply for all list items
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        # apply for string items, if not string return an empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

# apply clean function to the features
features = ['cast', 'keywords', 'director', 'genres']
for feature in features:
    df[feature] = df[feature].apply(clean_feature)

# pretty print features dataframe
from tabulate import tabulate
print(tabulate(df[['title', 'cast', 'director', 'keywords', 'genres']].head(5), headers='keys', tablefmt='psql'))

+----+------------------------------------------+-----------------------------------------------------+------------------+-------------------------------------------+-------------------------------------------+
|    | title                                    | cast                                                | director         | keywords                                  | genres                                    |
|----+------------------------------------------+-----------------------------------------------------+------------------+-------------------------------------------+-------------------------------------------|
|  0 | Avatar                                   | ['samworthington', 'zoesaldana', 'sigourneyweaver'] | jamescameron     | ['cultureclash', 'future', 'spacewar']    | ['action', 'adventure', 'fantasy']        |
|  1 | Pirates of the Caribbean: At World's End | ['johnnydepp', 'orlandobloom', 'keiraknightley']    | goreverbinski    | ['ocean', 'drugabuse', 'exoticisl

**Penjelasan kode program 2**
1. Mendefinisikan fungsi clean_feature untuk membersihkan data dengan:
Mengubah semua huruf menjadi lowercase.
Menghapus spasi dari setiap elemen list atau string.
Mengembalikan string kosong jika nilai bukan list atau string.
2. Menerapkan fungsi pembersih ke kolom penting (cast, keywords, director, genres) agar data lebih seragam dan konsisten.
3. Menampilkan data yang sudah dibersihkan dalam format tabel untuk memverifikasi hasil pembersihan.

Tujuan utamanya: membersihkan dan menormalkan teks data fitur agar siap digunakan untuk analisis teks atau perbandingan yang sensitif terhadap format (misalnya dalam sistem rekomendasi).











In [ ]:
#3. ADD FEATURE COLUMN
def add_feature_col(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
df['features'] = df.apply(add_feature_col, axis=1)
df['features'].head(5)

**Penjelasan kode program 3**
1. Membuat fungsi add_feature_col untuk:
Menggabungkan data dari kolom keywords, cast, director, dan genres menjadi satu string panjang.
Pemisah antar elemen menggunakan spasi (' '), sehingga cocok untuk analisis teks.
2. Menerapkan fungsi ke setiap baris dalam DataFrame menggunakan .apply(..., axis=1) untuk menghasilkan kolom baru features.

3. Hasil akhir:
Setiap baris di kolom features berisi gabungan kata-kata deskriptif tentang film tersebut.

Tujuan utamanya: membuat satu kolom teks yang menyatukan semua informasi penting dari film untuk keperluan pemodelan teks dan perhitungan kemiripan.

In [ ]:
#4. CREATE MATRIX
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# create count matrix and cosine similarity
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['features'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# reset index of the data frame and construct reverse mapping
df = df.reset_index()
indices = pd.Series(df.index, index=df['title'])

**Penjelasan kode program 4**
1. Membuat objek CountVectorizer:
Mengubah teks dari kolom features menjadi representasi numerik (vektor kata), dengan mengabaikan stopwords (kata umum yang tidak bermakna seperti "the", "and", dll).

2. Menghasilkan count_matrix:
Matriks ini berisi frekuensi kemunculan setiap kata pada tiap film, yang menjadi dasar untuk mengukur kemiripan antar film.

3. Menghitung cosine_similarity:
Menghasilkan matriks kemiripan antar semua film berdasarkan fitur gabungan mereka.
Nilainya antara 0 (tidak mirip) sampai 1 (identik).

4. Menyusun ulang indeks DataFrame dan membuat mapping:
Reset indeks agar rapi dan konsisten.
Membuat indices, yaitu peta dari judul film ke indeks baris di DataFrame—berguna untuk pencarian cepat saat merekomendasikan film.

Tujuan utamanya: mengubah data teks film menjadi bentuk numerik, lalu menghitung tingkat kemiripan antar film, sebagai dasar sistem rekomendasi film berbasis konten.

In [ ]:
#5. FIND RECOMMENDATIONS
# function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim):
    # get the index of the movie that matches the title
    idx = indices[title]

    # get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # return the top 10 most similar movies from the data frame
    return df['title'].iloc[movie_indices]


# recommendatain of movie JFK
get_recommendations('JFK', cosine_sim)

# recommendatain of movie JFK
get_recommendations('The Godfather', cosine_sim)

# get recommendation of movie Avatar
get_recommendations('Avatar',cosine_sim)


**Penjelasan kode program 5**
1. Membuat fungsi get_recommendations(title, cosine_sim):
Input: judul film dan matriks kemiripan.
Output: daftar 10 film yang paling mirip dengan film yang diberikan.
2. Langkah-langkah dalam fungsi:
Menemukan indeks film berdasarkan judul.
Mengambil skor kemiripan (cosine similarity) antara film tersebut dan semua film lain.
Mengurutkan skor kemiripan secara menurun (mirip ke tidak mirip).
Mengambil 10 film teratas (kecuali film itu sendiri).
Mengembalikan judul-judul film yang paling mirip.
3. Menguji fungsi rekomendasi:
Menampilkan 10 film yang paling mirip dengan:
'JFK'
'The Godfather'
'Avatar'

Tujuan utamanya: membuat dan menjalankan sistem rekomendasi film berbasis konten, yang menyarankan film-film lain yang memiliki fitur serupa dengan film yang dipilih pengguna.








